<a href="https://colab.research.google.com/github/kit-jiyu/Branch-Repository/blob/main/%EC%8B%AC%EC%9E%A5%EB%B3%91_%EC%98%88%EC%A0%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#1.필요한 모듈 import
머신러닝에 필요한 파일, 함수들을 불러옵니다.


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, recall_score, precision_score
from sklearn.metrics import confusion_matrix
from sklearn.decomposition import PCA
import matplotlib.colors as colors


#이때 불러온 데이터를 기반으로 학습됨.
  #불러온 데이터가 차별적일 시 차별적인 모델이 만들어진다.
df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/heart.csv")
df.head()

#2. 데이터 전처리
원핫인코딩, 정규화, training,test 세트 분리

In [ ]:
#데이터를 컴퓨터가 이해할 수 있도록 수치화
a = pd.get_dummies(df['cp'], prefix = "cp")
b = pd.get_dummies(df['thal'], prefix = "thal")
c = pd.get_dummies(df['slope'], prefix = "slope")
frames = [df, a, b, c]
df = pd.concat(frames, axis = 1)
df = df.drop(columns = ['cp', 'thal', 'slope'])
df.head()
x_data = df.drop(['target'], axis = 1)
x = (x_data - np.min(x_data)) / (np.max(x_data) - np.min(x_data)).values
y = df.target.values

#데이터를 train용과 test용 으로 분할: 수능준비에 있어 문제집과 실전모의고사와 같은 개념.
  #train용: 모델 학습에 사용
  #test용: 모델 평가에 사용(제대로 만들어졌나)
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size = 0.2,random_state=0)

#3.모델링
SVM, Deep Learning과 같은 알고리즘을 사용하여 모델을 생성합니다

In [ ]:
svm = SVC(random_state = 42)

#4.모델학습
생성된 모델을 학습시킵니다.

In [ ]:
#지도학습의 경우 train 할때 문제와 정답을 같이 준다.
svm.fit(x_train, y_train)

#5. 모델평가
모델이 잘 만들어졌는지 평가합니다.

In [ ]:
#대표적인 평가지표들
def print_arp(y_head_svm):
  acc = accuracy_score(y_test, y_head_svm)
  rec = recall_score(y_test, y_head_svm)
  prec= precision_score(y_test, y_head_svm)
  print("Test Accuracy: {:.2f}".format(acc))
  print("Test recall: {:.2f}".format(rec))
  print("Test Precision: {:.2f}".format(prec))
  print()
  return acc,rec,prec

y_head_svm = svm.predict(x_test)
print_arp(y_head_svm)


#confusion_matrix 그려서 파악
def draw_confusion_matrix(y_head_svm,n1):
  cm_svm = confusion_matrix(y_test,y_head_svm)
  plt.figure(figsize=(12,6))
  plt.subplot(2,3,n1)
  plt.title("Support Vector Machine Confusion Matrix")
  sns.heatmap(cm_svm,annot=True,annot_kws={"size": 24},fmt="d",cmap="Blues",cbar=False,xticklabels = ['Predicted positive','Predicted negative'],yticklabels=['Actual positive','Actual negative'])

draw_confusion_matrix(y_head_svm,1)


#6.모델 활용


In [ ]:
pca = PCA()
X_train_pca = pca.fit_transform(x_train)

def svm_draw(svm):
  pc1 = X_train_pca[:, 0]
  pc2 = X_train_pca[:, 1]
  svm.fit(np.column_stack((pc1, pc2)), y_train)
  x_min = pc1.min() - 1
  x_max = pc1.max() + 1
  y_min = pc2.min() - 1
  y_max = pc2.max() + 1
  xx, yy = np.meshgrid(np.arange(start=x_min, stop=x_max, step=0.1),
                      np.arange(start=y_min, stop=y_max, step=0.1))


  #임의의 데이터 생성
  arg = np.column_stack((xx.ravel(), yy.ravel()))
  print("투입된 데이터:")
  print(arg)

  #임의의 데이터를 모델에 투입하여 예측
  Z = svm.predict(arg)
  Z = Z.reshape(xx.shape)


  fig, ax = plt.subplots(figsize=(3,3))
  ax.contourf(xx, yy, Z, alpha=0.1)
  cmap = colors.ListedColormap(['#e41a1c', '#4daf4a'])
  scatter = ax.scatter(pc1, pc2, c=y_train,
                cmap=cmap,
                s=100,
                edgecolors='k',
                alpha=0.7)
  legend = ax.legend(scatter.legend_elements()[0],
                    scatter.legend_elements()[1],
                      loc="upper right")
  legend.get_texts()[0].set_text("No HD")
  legend.get_texts()[1].set_text("Yes HD")
  ax.set_ylabel('PC2')
  ax.set_xlabel('PC1')
  ax.set_title('Decison surface using the PCA transformed/projected features')
  plt.show()


print()
svm_draw(svm)
